In [ ]:
! pip install chembl_webresource_client
! pip install lazypredict
! pip install jcopml==1.1.1

In [ ]:
!pip install rdkit

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score
# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz
import pandas as pd
import numpy as np
from chembl_webresource_client.new_client import new_client
#from jcopml.plot import plot_missing_value

import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from rdkit.Chem import Descriptors, Lipinski

import seaborn as sns
sns.set(style='ticks')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

import lazypredict
from lazypredict.Supervised import LazyRegressor, LazyClassifier

from tqdm import tqdm

#from jcopml.pipeline import num_pipe, cat_pipe
#from jcopml.utils import save_model, load_model
#from jcopml.plot import plot_missing_value
#from jcopml.feature_importance import mean_score_decrease

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


# Import Dataset

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/02_dataset_model_sars.csv')
df1

,0,1,2,3,4,5,6,7,8,9,...,159,160,161,162,163,pIC50,MW,LogP,NumHDonors,NumHAcceptors
0,0,1,1,0,1,0,0,1,1,1,...,0,0,0,0,0,4.75,430.35,4.69,3.00,6.00
1,0,1,1,0,1,0,0,1,1,1,...,0,1,0,0,0,4.78,364.34,3.93,3.00,5.00
2,0,1,1,0,1,0,0,1,1,1,...,0,0,0,0,0,4.00,431.46,3.63,3.00,7.00
3,1,1,1,0,0,0,0,1,1,1,...,0,0,0,0,0,4.05,364.34,3.93,3.00,5.00
4,0,1,1,0,0,0,0,1,1,1,...,0,0,0,0,0,4.00,366.43,3.85,3.00,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,1,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,7.31,475.86,5.31,4.00,6.00
771,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,6.46,459.41,4.80,4.00,6.00
772,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,7.30,407.87,4.29,4.00,6.00
773,1,1,0,0,0,0,0,1,1,1,...,0,0,0,0,0,7.28,388.44,4.14,2.00,8.00


In [ ]:
df1.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '159', '160', '161', '162', '163', 'pIC50', 'MW', 'LogP', 'NumHDonors',
       'NumHAcceptors'],
      dtype='object', length=169)

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/01_label2class_sars.csv')
df2

,molecule_chembl_id,class,canonical_smiles,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL2332869,inactive,O=C(Nc1ccc(Oc2ncnc3[nH]ncc23)cc1)Nc1cccc(OC(F)...,430.35,4.69,3.00,6.00,4.75
1,CHEMBL2332868,inactive,O=C(Nc1ccc(Oc2ncnc3[nH]ncc23)cc1)Nc1cccc(F)c1,364.34,3.93,3.00,5.00,4.78
2,CHEMBL2332866,inactive,O=C(Nc1ccc(Oc2ncnc3[nH]ncc23)cc1)Nc1ccc(N2CCOC...,431.46,3.63,3.00,7.00,4.00
3,CHEMBL2332865,inactive,O=C(Nc1ccc(F)cc1)Nc1ccc(Oc2ncnc3[nH]ncc23)cc1,364.34,3.93,3.00,5.00,4.05
4,CHEMBL2332864,inactive,O=C(NCC1CCCCC1)Nc1ccc(Oc2ncnc3[nH]ncc23)cc1,366.43,3.85,3.00,5.00,4.00
...,...,...,...,...,...,...,...,...
770,CHEMBL412102,active,Nc1ncnc2c1N=C(c1ccc(NC(=O)Nc3ccc(Cl)c(C(F)(F)F...,475.86,5.31,4.00,6.00,7.31
771,CHEMBL429516,active,Nc1ncnc2c1N=C(c1ccc(NC(=O)Nc3ccc(F)c(C(F)(F)F)...,459.41,4.80,4.00,6.00,6.46
772,CHEMBL409872,active,Nc1ncnc2c1N=C(c1ccc(NC(=O)Nc3cccc(Cl)c3)cc1)CCN2,407.87,4.29,4.00,6.00,7.30
773,CHEMBL261592,active,Cc1cc2cc(Nc3ncnn4cc(-c5nnc(C)o5)c(C(C)C)c34)cn...,388.44,4.14,2.00,8.00,7.28


In [ ]:
df2.columns

Index(['molecule_chembl_id', 'class', 'canonical_smiles', 'MW', 'LogP',
       'NumHDonors', 'NumHAcceptors', 'pIC50'],
      dtype='object')

## Merging

In [ ]:
df_combined = pd.concat([df1,df2], axis=1)
df_combined

,0,1,2,3,4,5,6,7,8,9,...,NumHDonors,NumHAcceptors,molecule_chembl_id,class,canonical_smiles,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,0,1,1,0,1,0,0,1,1,1,...,3.00,6.00,CHEMBL2332869,inactive,O=C(Nc1ccc(Oc2ncnc3[nH]ncc23)cc1)Nc1cccc(OC(F)...,430.35,4.69,3.00,6.00,4.75
1,0,1,1,0,1,0,0,1,1,1,...,3.00,5.00,CHEMBL2332868,inactive,O=C(Nc1ccc(Oc2ncnc3[nH]ncc23)cc1)Nc1cccc(F)c1,364.34,3.93,3.00,5.00,4.78
2,0,1,1,0,1,0,0,1,1,1,...,3.00,7.00,CHEMBL2332866,inactive,O=C(Nc1ccc(Oc2ncnc3[nH]ncc23)cc1)Nc1ccc(N2CCOC...,431.46,3.63,3.00,7.00,4.00
3,1,1,1,0,0,0,0,1,1,1,...,3.00,5.00,CHEMBL2332865,inactive,O=C(Nc1ccc(F)cc1)Nc1ccc(Oc2ncnc3[nH]ncc23)cc1,364.34,3.93,3.00,5.00,4.05
4,0,1,1,0,0,0,0,1,1,1,...,3.00,5.00,CHEMBL2332864,inactive,O=C(NCC1CCCCC1)Nc1ccc(Oc2ncnc3[nH]ncc23)cc1,366.43,3.85,3.00,5.00,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,1,1,0,0,1,0,1,0,0,0,...,4.00,6.00,CHEMBL412102,active,Nc1ncnc2c1N=C(c1ccc(NC(=O)Nc3ccc(Cl)c(C(F)(F)F...,475.86,5.31,4.00,6.00,7.31
771,1,1,0,0,0,0,1,0,0,0,...,4.00,6.00,CHEMBL429516,active,Nc1ncnc2c1N=C(c1ccc(NC(=O)Nc3ccc(F)c(C(F)(F)F)...,459.41,4.80,4.00,6.00,6.46
772,1,1,0,0,1,0,0,0,0,0,...,4.00,6.00,CHEMBL409872,active,Nc1ncnc2c1N=C(c1ccc(NC(=O)Nc3cccc(Cl)c3)cc1)CCN2,407.87,4.29,4.00,6.00,7.30
773,1,1,0,0,0,0,0,1,1,1,...,2.00,8.00,CHEMBL261592,active,Cc1cc2cc(Nc3ncnn4cc(-c5nnc(C)o5)c(C(C)C)c34)cn...,388.44,4.14,2.00,8.00,7.28
